In [17]:
from sklearn.datasets import load_files       
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    kaddi_names = [item[8:-1] for item in sorted(glob(path+"/*/"))]
    data = load_files(path)
    kaddi_files = np.array(data['filenames'])
    kaddi_targets = np_utils.to_categorical(np.array(data['target']), len(kaddi_names))
    return kaddi_files, kaddi_targets, kaddi_names;

In [18]:
kaddi_files, kaddi_targets, kaddi_names = load_dataset('Dataset')

In [19]:
kaddi_files

array(['Dataset/bad_sticks/b655.jpg', 'Dataset/good_sticks/g219.jpg',
       'Dataset/good_sticks/g1877.jpg', ...,
       'Dataset/good_sticks/g1373.jpg', 'Dataset/good_sticks/g2231.jpg',
       'Dataset/good_sticks/g2344.jpg'], 
      dtype='<U29')

In [20]:
kaddi_targets

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [21]:
kaddi_names

['bad_sticks', 'good_sticks']

In [22]:
train_files, test_files, train_targets, test_targets = train_test_split(kaddi_files, kaddi_targets, test_size=0.2, random_state=0)

train_files, valid_files, train_targets, valid_targets = train_test_split(train_files, train_targets, test_size=0.2, random_state=0)

# # print statistics about the dataset
print('There are %d total kaddi categories.' % len(kaddi_names))
print('There are %s total kaddi images.\n' % len(kaddi_files))
print('There are %d training kaddi images.' % len(train_files))
print('There are %d validation kaddi images.' % len(valid_files))
print('There are %d test kaddi images.'% len(test_files))

There are 2 total kaddi categories.
There are 4500 total kaddi images.

There are 2880 training kaddi images.
There are 720 validation kaddi images.
There are 900 test kaddi images.


### Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [23]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224), grayscale=True)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [24]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 900/900 [00:10<00:00, 81.82it/s]


In [32]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Convolution2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

def getMyModel():
    model = Sequential()
    model.add(Convolution2D(filters=16, kernel_size=3,padding='same', activation='relu', input_shape=train_tensors.shape[1:]))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Convolution2D(32,2,padding='same',activation='relu'))
    model.add(MaxPooling2D(2))
#     model.add(Convolution2D(64,2,padding='same',activation='relu'))
#     model.add(MaxPooling2D(2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(len(kaddi_names), activation='softmax'))
    model.summary()
    return model

In [33]:
model = getMyModel()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 16)      160       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total params: 2,306.0
Trainable params: 2,306.0
Non-trainable params: 0.0
_________________________________________________________________


### Compile the Model

In [34]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [35]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 12

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 2880 samples, validate on 720 samples
Epoch 1/12
2880/2880 [==============================] - 77s - loss: 0.5947 - acc: 0.7267 - val_loss: 0.5999 - val_acc: 0.7153
Epoch 2/12
2880/2880 [==============================] - 73s - loss: 0.5902 - acc: 0.7267 - val_loss: 0.5963 - val_acc: 0.7153
Epoch 3/12
2880/2880 [==============================] - 72s - loss: 0.5869 - acc: 0.7267 - val_loss: 0.5931 - val_acc: 0.7153
Epoch 4/12
2880/2880 [==============================] - 72s - loss: 0.5824 - acc: 0.7267 - val_loss: 0.5897 - val_acc: 0.7153
Epoch 5/12
2880/2880 [==============================] - 73s - loss: 0.5730 - acc: 0.7257 - val_loss: 0.5741 - val_acc: 0.7153
Epoch 6/12
2880/2880 [==============================] - 72s - loss: 0.5555 - acc: 0.7330 - val_loss: 0.5503 - val_acc: 0.7222
Epoch 7/12
2880/2880 [==============================] - 73s - loss: 0.5276 - acc: 0.7524 - val_loss: 0.5127 - val_acc: 0.7653
Epoch 8/12
2880/2880 [==============================] - 72s - loss: 0.5

### Load the Model with the Best Validation Loss

In [36]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [37]:
# get index of predicted dog breed for each image in test set
kaddi_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(kaddi_predictions)==np.argmax(test_targets, axis=1))/len(kaddi_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 84.1111%


In [38]:
model.save('a.model')